# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [32]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

### 데이터 셋 읽어오기

In [33]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [34]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


## 2. 데이터 전처리

### 레이블 인코딩

In [35]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [36]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

다시 학습 데이터와 제출 데이터를 분리합니다.

In [37]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]
#df_train.dropna(axis=1, inplace=True)
#df_test.dropna(axis=1, inplace=True)

### VIF 다중공선성 해결

In [38]:
from sklearn.linear_model import LinearRegression as LR
#df_train.drop(['customer_country.1'],axis=1,inplace=True)
#df_test.drop(['customer_country.1','idit_strategic_ver'],axis=1,inplace=True)
VIF_dict = dict()
df_train.fillna(0,inplace=True)
df_test.fillna(0,inplace=True)
for col in df_train.columns:
    model = LR().fit(df_train.drop([col], axis=1), df_train[col])
    r2 = model.score(df_train.drop([col], axis=1), df_train[col])
    VIF = 1 / max((1 - r2), 1e-9)
    VIF_dict[col] = VIF

# VIF 값이 5 이상인 열을 출력
high_VIF_columns = [col for col, vif in VIF_dict.items() if vif >= 5]
print("Columns with VIF >= 5:", high_VIF_columns)
df_train.drop(['customer_country', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'customer_country.1'],axis=1,inplace=True)
df_test.drop(['customer_country', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'customer_country.1'],axis=1,inplace=True)

Columns with VIF >= 5: ['customer_country', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'customer_country.1']


### 2-2. 학습, 검증 데이터 분리

#### tomek links

In [39]:
'''
# 수치형 컬럼의 NaN 값을 평균으로 대체
# 수치형 컬럼을 식별합니다.
numeric_columns = list(set(df_train.columns) - set(label_columns))

for col in numeric_columns:
    if df_train[col].isnull().any():
        df_train[col].fillna(df_train[col].mean(), inplace=True)
categorical_features_indices=[]
# 범주형 컬럼의 NaN 값을 가장 빈번한 값으로 대체
for col in label_columns:
    if df_train[col].isnull().any():
        df_train[col].fillna(df_train[col].mode()[0], inplace=True)
    categorical_features_indices=df_train.columns.get_loc(col)
    
'''

'\n# 수치형 컬럼의 NaN 값을 평균으로 대체\n# 수치형 컬럼을 식별합니다.\nnumeric_columns = list(set(df_train.columns) - set(label_columns))\n\nfor col in numeric_columns:\n    if df_train[col].isnull().any():\n        df_train[col].fillna(df_train[col].mean(), inplace=True)\ncategorical_features_indices=[]\n# 범주형 컬럼의 NaN 값을 가장 빈번한 값으로 대체\nfor col in label_columns:\n    if df_train[col].isnull().any():\n        df_train[col].fillna(df_train[col].mode()[0], inplace=True)\n    categorical_features_indices=df_train.columns.get_loc(col)\n    \n'

In [40]:

# TRUE와 FALSE 개수 세기
true_count = df_train['is_converted'].sum()
false_count = len(df_train) - true_count

# 두 개수 중 작은 값으로 데이터를 분할
min_count = min(true_count, false_count)

# TRUE와 FALSE 개수를 맞추어 데이터를 분할
true_data = df_train[df_train['is_converted'] == True].sample(n=min_count, random_state=400)
false_data = df_train[df_train['is_converted'] == False].sample(n=min_count, random_state=400)

# 데이터를 결합
df_balanced = pd.concat([true_data, false_data])

# val set을 먼저 구성
val_size = int(len(df_balanced) * 0.2)  # 전체 데이터의 20%를 val set으로 사용
val_set = df_balanced.sample(n=val_size, random_state=400)

# val_set을 x_val과 y_val로 분리
x_val = val_set.drop("is_converted", axis=1)
y_val = val_set["is_converted"]

# train set 구성 (비율을 맞추기 전의 데이터 사용)
train_set = df_train.drop(val_set.index)

# train set과 val set 구성 확인
print("Train set:")
print(train_set['is_converted'].value_counts())
print("Validation set:")
print(val_set['is_converted'].value_counts())


Train set:
is_converted
False    53483
True      3876
Name: count, dtype: int64
Validation set:
is_converted
True     974
False    966
Name: count, dtype: int64


In [41]:

from imblearn.under_sampling import TomekLinks, RandomUnderSampler
import pandas as pd


# Tomek Link를 적용하여 데이터 보정
tl = TomekLinks()
x_train_resampled, y_train_resampled = tl.fit_resample(train_set.drop("is_converted", axis=1), train_set["is_converted"])

# 클래스 비율 맞추기
rus = RandomUnderSampler(sampling_strategy={0: int(y_train_resampled.sum() * 1), 1: y_train_resampled.sum()}, random_state=42)
x_train, y_train = rus.fit_resample(x_train_resampled, y_train_resampled)

# 결과 확인
print("Train set after Tomek Links and balancing classes:")
print(pd.Series(y_train).value_counts())


Train set after Tomek Links and balancing classes:
is_converted
False    3876
True     3876
Name: count, dtype: int64


In [42]:
# x_train, x_val, y_train, y_val = train_test_split(
#     df_train.drop("is_converted", axis=1),
#     df_train["is_converted"],
#     test_size=0.2,
#     shuffle=True,
#     random_state=400,
# )

## 3. 모델 학습

### 모델 정의 

In [43]:
#model = DecisionTreeClassifier()
#model = DecisionTreeClassifier(max_depth=12, min_samples_split=3, min_samples_leaf=5)
#model = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=2)
true_count = sum(df_train['is_converted'] == True)
false_count = sum(df_train['is_converted'] == False)
scale_pos_weight = false_count / true_count
#scale_pos_weight=scale_pos_weight*0.5
# XGBoost 모델 초기화
model = XGBClassifier()


model_param_grid = {
    'n_estimators': [100, 150,200],
    'learning_rate': [0.1, 0.15, 0.2, 0.3],
    'max_depth': [6, 8, 10, 12]
}

model_grid=GridSearchCV(model, param_grid = model_param_grid, scoring="accuracy", n_jobs=-1, verbose = 1)
model_grid.fit(x_train, y_train)
model = model_grid.best_estimator_
model.fit(x_train, y_train) 





Fitting 5 folds for each of 48 candidates, totalling 240 fits


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

### 모델 학습

In [44]:
model.fit(x_train.fillna(0), y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

### 모델 성능 보기

In [45]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, pred)
    TP = confusion[1][1]
    FP = confusion[0][1]
    TN = confusion[0][0]
    FN = confusion[1][0]
    print(f"True Positive(TP): {TP}, False Positive(FP): {FP}, True Negative(TN): {TN}, False Negative(FN): {FN}")
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [46]:
#pred = model.predict(x_val_selected_features.fillna(0))
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)
print(type(model))
#print("트리의 최대 깊이:", model.tree_.max_depth)


True Positive(TP): 920, False Positive(FP): 67, True Negative(TN): 899, False Negative(FN): 54
오차행렬:
 [[899  67]
 [ 54 920]]

정확도: 0.9376
정밀도: 0.9321
재현율: 0.9446
F1: 0.9383
<class 'xgboost.sklearn.XGBClassifier'>


## 4. 제출하기

### 테스트 데이터 예측

In [47]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [48]:
test_pred = model.predict(x_test.fillna(0))
sum(test_pred) # True로 예측된 개수

2027

### 제출 파일 작성

In [49]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

#in vscode gitignore issue

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**